In [1]:
%cd ..

/data/home/eak/learning/llm_finetuning/specializing-llm-telecom


In [2]:
import os

os.environ["TOKENIZERS_PARALLELISM"] = "true"
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

import json

import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

MODEL_PATH = "unsloth/llama-3-8b-Instruct-bnb-4bit"
# MODEL_PATH = "unsloth/tinyllama-bnb-4bit"

In [3]:
from unsloth import FastLanguageModel
import torch
from trl import SFTTrainer
from transformers import TrainingArguments, AutoTokenizer
from peft import PeftModelForCausalLM

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


In [4]:
max_seq_length = 4096 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.


result: tuple[PeftModelForCausalLM, AutoTokenizer] = FastLanguageModel.from_pretrained(
	model_name = MODEL_PATH,
	max_seq_length = max_seq_length,
	dtype = dtype,
	load_in_4bit = load_in_4bit,
	# token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)
model: PeftModelForCausalLM = result[0]
tokenizer: AutoTokenizer = result[1]

==((====))==  Unsloth: Fast Llama patching release 2024.5
   \\   /|    GPU: NVIDIA RTX A6000. Max memory: 47.438 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.3.1+cu118. CUDA = 8.6. CUDA Toolkit = 11.8.
\        /    Bfloat16 = TRUE. Xformers = 0.0.26.post1+cu118. FA = False.
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [5]:
import json
question_format = json.dumps({
    "question 0": {
		"question": "What is the purpose of the Nmfaf_3daDataManagement_Deconfigure service operation? [3GPP Release 18]",
		"option 1": "To configure the MFAF to map data or analytics received by the MFAF to out-bound notification endpoints",
		"option 2": "To configure the MFAF to stop mapping data or analytics received by the MFAF to out-bound notification endpoints",
		"option 3": "To supply data or analytics from the MFAF to notification endpoints",
		"option 4": "To fetch data or analytics from the MFAF based on fetch instructions",
		"answer": "option 2: To configure the MFAF to stop mapping data or analytics received by the MFAF to out-bound notification endpoints",
		"explanation": "The Nmfaf_3daDataManagement_Deconfigure service operation is used to stop mapping data or analytics received by the MFAF to one or more out-bound notification endpoints.",
		"category": "Standards specifications"
	}
})

QUERY = """Below you have context about 5G and 3GPP (3rd Generation Partnership Project) standard, as a usefull AI Model you will generate 5 questions in a QCM style following this format
{question_format}

### Context
{context}

### Response
"""

In [6]:
context = """
Foreword

This Technical Specification has been produced by the 3rd Generation Partnership Project (3GPP).

The contents of the present document are subject to continuing work within the TSG and may change following formal TSG approval. Should the TSG modify the contents of the present document, it will be re-released by the TSG with an identifying change of release date and an increase in version number as follows:

Version x.y.z

where:

x the first digit:

1 presented to TSG for information;

2 presented to TSG for approval;

3 or greater indicates TSG approved document under change control.

y the second digit is incremented for all changes of substance, i.e. technical enhancements, corrections, updates, etc.

z the third digit is incremented when editorial only changes have been incorporated in the document.

Introduction

In order to ensure correctness and consistency of the specifications (i.e., technical specifications and technical reports) under responsibility of the Technical Specification Groups (TSG) of the 3rd Generation Partnership Project (3GPP), clear, manageable and efficient mechanisms are necessary to handle version control, change control, document updating, distribution and management.

Also, the fact that the specifications are/will be implemented by industry almost in parallel with the writing of them requires strict and fast procedures for handling of changes to the specifications.

It is very important that the changes that are brought into the standard, from the past, at present and in the future, are well documented and controlled, so that technical consistency and backwards tracing are ensured.

The 3GPP TSGs, and their sub-groups together with the Support Team are responsible for the technical content and consistency of the specifications whilst the Support Team alone is responsible for the proper management of the entire documentation, including specifications, meeting documents, administrative information and information exchange with other bodies.

3GPP

3GPP TR 21.900 V18.1.0 (2023-09)

14

Release 18

1 Scope

This document outlines the working methods to be used by the 3GPP Technical Specification Groups and their Working Groups and their Sub-Groups, and by the 3GPP Support Team in relation to document management, i.e. handling of specifications, updating procedures, Change Request procedures, version control mechanisms, specifications status information etc. It complements the rules and procedures defined for 3GPP. This document does not stipulate the details of the internal working of the TSG Sub-Groups. From the Technical Specification Group point of view, a task and responsibility is given to a Working Group directly answering to the Technical Specification Group. In practice, the work/task may be carried out in a subgroup of that Working Group.

1A References

The following documents contain provisions which, through reference in this text, constitute provisions of the present document.

- References are either specific (identified by date of publication, edition number, version number, etc.) or nonspecific.

- For a specific reference, subsequent revisions do not apply.

- For a non-specific reference, the latest version applies. In the case of a reference to a 3GPP document (including a GSM document), a non-specific reference implicitly refers to the latest version of that document in the same Release as the present document.

[1] 3GPP TR 21.801: "Specification drafting rules".

[2] 3GPP TR 21.905: "Vocabulary for 3GPP Specifications".

[3] 3GPP TS 21.101: "Technical Specifications and Technical Reports for a UTRAN-based 3GPP system".

[4] 3GPP TS 41.101: "Technical Specifications and Technical Reports for a GERAN-based 3GPP system".

[5] ITU-T Recommendation  I.130: "Method for the characterization of telecommunication services supported by an ISDN and network capabilities of an ISDN". 

[6] 3GPP TS 29.501: "5G System; Principles and Guidelines for Services Definition; Stage 3".

[7] IETF RFC 3629: "UTF-8, a transformation format of ISO 10646".

2 Definitions and abbreviations

For the purposes of the present document, the following terms and those in 3GPP TR 21.905 [2] apply.

building block: sub-division of a feature, representing a coherent set of technical functionality which would generally be expected to reside in a single system element.

change control: procedure whereby proposed modifications to a specification are presented for approval to the TSG as formal Change Requests.

closed: release status in which no changes of any kind to the specification are permitted.

Change Request (CR): formal proposal presented on a standard form to modify a specification which is under change control.

draft: specification status prior to change control, in which changes may be made without formal Change Requests.

early implementation: implementation of a particular feature on a platform of a release earlier than the release that contains the feature.

feature: new or substantially enhanced functionality which represents added value to the existing system.

frozen: release status in which only essential corrections are permitted.

functionality: normative text contained in one or more Technical Specifications, corresponding either to a feature or to some portion of a feature. 

group: TSG or TSG sub-group.

major version: For version x.y.z of a specification, x is called the major version.

Example: For version 3.2.0 of a specification, the major version is 3.

Mobile Competence Centre (MCC): The permanent secretariat, or support team, of 3GPP.

pseudo Change Request (pCR): similar to a Change Request but has no CR number and is intended to propose new or revised text for inclusion in 3GPP TSs or TRs not yet under change control (i.e. still in the drafting phase). Known in some groups as "text proposal".

specification: generic term standing for Technical Specification and Technical Report.

Study Item (SI): type of Work Item which will conduct feasibility studies and will result in a Technical Report

Study Item description (SID): description of a Study Item in a standard Work Item Description sheet.

TSG: Technical Specification Group.

TSG change control: specification status in which the Technical Specification Group is responsible for approval of Change Requests.

TSG sub-group: Working Group or subgroup of a Working Group or of a Sub-Group.

Working Group (WG): official subgroup of a TSG reporting to that TSG.

WG Change Control: specification status in which the Working Group is responsible for agreeing Change Requests for submission to the TSG for approval.

version: unique identifier in the form x.y.z for a specification at a given point in time.

Example: version 3.12.3.

withdrawn: specification status in which the given version of the specification no longer belongs to the appropriate set of valid specifications.

Work Item (WI): description of an enhancement to a technical area, which may be categorized as Study Item, Feature, Building Block or Work Task.

Work Item description (WID): description of a Work Item in a standard Work Item Description sheet.

work task: sub-division of a building block, representing a self-contained, well-scoped and well-scheduled item of work.

3 General responsibilities of the Support Team

3.1 Specifications, meetings and liaisons

The Support Team is responsible for the management of the work of the TSGs. This includes editorship and management of specifications once they have been put under TSG change control. It also includes preparation of and support for the meetings (including meeting reports) of the TSGs and their Working Groups, and subgroups in descending priority.

It furthermore includes liaison with other bodies and relevant groups and institutions.

3.2 Registration of code points

In the course of 3GPP's work, it will from time to time be necessary to register code points in protocols maintained by bodies other than 3GPP, for example, Multipurpose Internet Mail Extensions (MIME) types registered with the Internet Assigned Numbers Authority (IANA, http://www.iana.org/).

Wherever possible, registration of such code points shall be entrusted to the 3GPP Support Team rather than being performed by an individual delegate.  Since 3GPP is not a legally constituted entity, the Support Team shall register such code points in the name of one of the Organizational Partners on behalf of all the Organizational Partners of 3GPP.

4 Handling of Specifications

4.0 Numbering scheme

The specifications shall be numbered according to the following scheme:

3GPP TS aa.bbb (for Technical Specifications); or

3GPP TR aa.bbb (for Technical Reports).

The fields aa and bbb shall be selected according to the nature of the specification as given in tables 1 and 2. The provisions of table 1 shall be strictly enforced, but those of table 2 should be used for guidance: it is acceptable to deviate from these provisions for backwards compatibility or other reasons.
"""

In [7]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt=True)

In [8]:
text = [
    QUERY.format(question_format=question_format, context=context)
]
len(text)

1

In [9]:
inputs = tokenizer(
text, return_tensors = "pt", padding=True, truncation=True).to("cuda")

In [10]:
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 1024,)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Here are 5 questions in a QCM style based on the provided context:

{"question 0": {"question": "What is the purpose of the Nmfaf_3daDataManagement_Deconfigure service operation?", "option 1": "To configure the MFAF to map data or analytics received by the MFAF to out-bound notification endpoints", "option 2": "To configure the MFAF to stop mapping data or analytics received by the MFAF to out-bound notification endpoints", "option 3": "To supply data or analytics from the MFAF to notification endpoints", "option 4": "To fetch data or analytics from the MFAF based on fetch instructions", "answer": "option 2: To configure the MFAF to stop mapping data or analytics received by the MFAF to out-bound notification endpoints", "explanation": "The Nmfaf_3daDataManagement_Deconfigure service operation is used to stop mapping data or analytics received by the MFAF to one or more out-bound notification endpoints.", "category": "Standards specifications"}}

{"question 1": {"question": "What is th

In [11]:
print(tokenizer.batch_decode(_)[0])

<|begin_of_text|>Below you have context about 5G and 3GPP (3rd Generation Partnership Project) standard, as a usefull AI Model you will generate 5 questions in a QCM style following this format
{"question 0": {"question": "What is the purpose of the Nmfaf_3daDataManagement_Deconfigure service operation? [3GPP Release 18]", "option 1": "To configure the MFAF to map data or analytics received by the MFAF to out-bound notification endpoints", "option 2": "To configure the MFAF to stop mapping data or analytics received by the MFAF to out-bound notification endpoints", "option 3": "To supply data or analytics from the MFAF to notification endpoints", "option 4": "To fetch data or analytics from the MFAF based on fetch instructions", "answer": "option 2: To configure the MFAF to stop mapping data or analytics received by the MFAF to out-bound notification endpoints", "explanation": "The Nmfaf_3daDataManagement_Deconfigure service operation is used to stop mapping data or analytics received 